In [1]:
import numpy as np
import torch
from torch.autograd import Variable
import pandas as pd

In [2]:
# Read in training data
train_data_df = pd.read_csv("train_data.csv", header=0, index_col=0)
train_labels_df = pd.read_csv("train_labels.csv", header=0, index_col=0)
print(train_data_df.head())
print(train_labels_df.head())

   in_degree  out_degree  num_descendants  longest_descendant_path  \
0          0           2                4                        2   
1          0           1                2                        1   
2          1           1                2                        1   
3          2           0                1                        0   
4          1           0                1                        0   

   in_degree_betweenness_centrality  out_degree_betweenness_centrality  \
0                               0.0                                0.4   
1                               0.0                                0.2   
2                               0.2                                0.2   
3                               0.4                                0.0   
4                               0.2                                0.0   

   trophic_levels  
0             1.0  
1             1.0  
2             2.0  
3             2.0  
4             3.0  
      psize
0 

In [3]:
# Convert to numpy arrays
train_data = train_data_df.to_numpy()
train_labels = train_labels_df.to_numpy()

print(len(train_data))
print(train_data[:2])
print(len(train_labels))
print(train_labels[:2])

1301
[[0.  2.  4.  2.  0.  0.4 1. ]
 [0.  1.  2.  1.  0.  0.2 1. ]]
1301
[[2.9999972 ]
 [1.99998992]]


In [4]:
class LinearRegression(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegression, self).__init__()
        self.linear = torch.nn.Linear(input_size, output_size)

    def forward(self, x):
        out = self.linear(x)
        return out

In [5]:
# Hyperparameters and initializations
input_dim = 7
output_dim = 1
lr = 0.01
epochs = 100

model = LinearRegression(input_dim, output_dim)
model = model.float()
criterion1 = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
print(model)

LinearRegression(
  (linear): Linear(in_features=7, out_features=1, bias=True)
)


In [6]:
inputs = Variable(torch.from_numpy(train_data).float())
labels = Variable(torch.from_numpy(train_labels).float())
#inputs
labels

tensor([[3.0000],
        [2.0000],
        [2.0000],
        ...,
        [2.0000],
        [1.0000],
        [1.0000]])

In [7]:
# Training loop 1
for epoch in range(epochs):

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer.zero_grad()

    # get output from the model, given the inputs
    outputs = model(inputs)

    # get loss for the predicted output
    loss = criterion1(outputs, labels)
    print(loss)
    # get gradients w.r.t to parameters
    loss.backward()

    # update parameters
    optimizer.step()

    print('epoch {}, loss {}'.format(epoch, loss.item()))

tensor(20.1181, grad_fn=<MseLossBackward0>)
epoch 0, loss 20.11812973022461
tensor(11.2661, grad_fn=<MseLossBackward0>)
epoch 1, loss 11.266084671020508
tensor(7.0691, grad_fn=<MseLossBackward0>)
epoch 2, loss 7.069088935852051
tensor(5.0749, grad_fn=<MseLossBackward0>)
epoch 3, loss 5.074882984161377
tensor(4.1233, grad_fn=<MseLossBackward0>)
epoch 4, loss 4.123287200927734
tensor(3.6654, grad_fn=<MseLossBackward0>)
epoch 5, loss 3.6653757095336914
tensor(3.4414, grad_fn=<MseLossBackward0>)
epoch 6, loss 3.4413998126983643
tensor(3.3284, grad_fn=<MseLossBackward0>)
epoch 7, loss 3.3284342288970947
tensor(3.2683, grad_fn=<MseLossBackward0>)
epoch 8, loss 3.2682886123657227
tensor(3.2334, grad_fn=<MseLossBackward0>)
epoch 9, loss 3.233410120010376
tensor(3.2107, grad_fn=<MseLossBackward0>)
epoch 10, loss 3.210745334625244
tensor(3.1941, grad_fn=<MseLossBackward0>)
epoch 11, loss 3.1941025257110596
tensor(3.1805, grad_fn=<MseLossBackward0>)
epoch 12, loss 3.1805405616760254
tensor(3.1687

In [8]:
print(model(inputs[:1]))
print(labels[:1])

tensor([[2.7108]], grad_fn=<AddmmBackward0>)
tensor([[3.0000]])


In [9]:
input_dim = 7
output_dim = 1
lr = 0.001
epochs = 100
criterion2 = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [10]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from graph_util.visualization_util import make_graph_visual
from scheduling_util.modified_etf import Mod_ETF
from scheduling_util.consolidated_functions import opt_schedule_given_ordering
from scheduling_util.heuristics import native_rescheduler
from graph_util.random_graph_functions import random_all_fork, random_all_join
from graph_util.erdos_renyi_dag import er_dag
from scheduling_util.approx_pseudosizes import speed_to_psize

In [11]:
# Helper Functions
feature_id = ['in_degree', 'out_degree', 'num_descendants', 'longest_descendant_path', 'in_degree_betweenness_centrality', 'out_degree_betweenness_centrality', 'trophic_levels']    

def get_feature_set(G, psize):
    
    lst = []
    in_degree_lst = G.in_degree
    out_degree_lst = G.out_degree
    in_bet_lst = nx.algorithms.in_degree_centrality(G)
    out_bet_lst = nx.algorithms.out_degree_centrality(G)
    trophic_lst = nx.algorithms.trophic_levels(G)
    for node in G.nodes:
        in_degree = in_degree_lst[node]
        out_degree = out_degree_lst[node]
        in_bet = in_bet_lst[node]
        # print("in betweennes", in_bet)
        out_bet = out_bet_lst[node]
        trophic = trophic_lst[node]
        # print("out_betweennes", out_bet)
        descendants = list(nx.algorithms.dag.descendants(G, node))
        descendants.append(node)
        num_descendants = len(descendants) 
        longest_descendant_path = nx.dag_longest_path_length(G.subgraph(descendants)) 
        lst.append(np.asarray([in_degree, out_degree, num_descendants, longest_descendant_path, in_bet, out_bet, trophic, psize[node]]))      
    return np.asarray(lst)

def compute_cost(w, t, s):
    '''
    Given weights w, time intervals t, and speeds s, compute the cost of the schedule; 
    returns total cost as well as separate power and time components.
    '''
    power = 0
    time = 0
    # print(t)
    for j in range(len(s)):
        if t[j] == -1:
            return -1, -1, -1
        power += w[j] * s[j]
        time += t[j][1]
    total_cost = power + time
    return total_cost, power, time

def psize_to_speed(psize):
    return [np.sqrt(psize[i])for i in range(len(psize))]

In [12]:
# Make the testing set
data = []
lengths = []
tie_breaking_rule = 2
graphs = []
x_tests = []
x_test_labels = []
greedy_objectives = []
probability = 0.3

for i in range(1, 20):
    print("I is ", i)
    for k in range(5):  
        num_tasks = i + 5
        num_machines = 3
        w = [1 for _ in range(num_tasks)]
        s = [1 for _ in range(num_tasks)]
        p = [1 for _ in range(num_tasks)]
        G, _ = er_dag(num_tasks, probability)
        assert(nx.algorithms.dag.is_directed_acyclic_graph(G))
        etf = Mod_ETF(G, w, s, num_machines, tie_breaking_rule, plot=False)
        intervals, speeds, obj_value = opt_schedule_given_ordering(True, G, w, p, etf.order, plot=False, compare=False)
        psize = speed_to_psize(speeds)
        cost, power, time = compute_cost(w, intervals, speeds)
        if cost != -1:
            graphs.append(G)
            greedy_objectives.append(np.asarray([cost, power, time]))
            x_tests.append(np.asarray(get_feature_set(G, psize)))
            x_test_labels.append(np.asarray(psize))


I is  1
I is  2
I is  3
I is  4
I is  5
I is  6
I is  7
I is  8
I is  9
I is  10
I is  11
I is  12
I is  13
I is  14
I is  15
I is  16
I is  17
I is  18
I is  19


In [23]:
#x_tests[:2]
print(x_test_labels)

[array([2.00001821, 2.00001821, 1.        , 1.        , 1.99998992,
       1.        ]), array([2.9999972, 2.9999972, 4.       , 1.       , 1.       , 1.       ]), array([1.99993336, 2.00007478, 1.        , 1.        , 1.99996164,
       1.        ]), array([2.5000037 , 1.99998992, 1.        , 2.5000037 , 1.        ,
       1.        ]), array([3.00120976, 2.0004425 , 1.        , 1.9995374 , 2.99878489,
       1.        ]), array([2.3333479 , 2.3333479 , 1.49998807, 2.3333479 , 1.49998807,
       1.        , 1.        ]), array([3.50000489, 3.50000489, 5.00000904, 2.00001821, 1.        ,
       1.99996164, 1.        ]), array([1.49998807, 4.        , 1.49998807, 2.9999972 , 1.        ,
       1.99998992, 1.        ]), array([1.        , 2.9999972 , 2.9999972 , 1.99998992, 1.99998992,
       1.99998992, 1.        ]), array([1.99998992, 1.        , 2.9999972 , 1.99998992, 1.99998992,
       1.        , 1.        ]), array([2.999824  , 3.00017041, 1.99998992, 1.99976366, 1.        ,
     

In [27]:
# Convert to numpy array
np.save('test_data', np.asarray(x_tests, dtype='object'))
np.save('test_labels', np.asarray(x_test_labels,dtype='object'))
np.save('test_objectives', np.asarray(greedy_objectives,dtype='object'))

In [ ]:
x_te